In [493]:
#We will schedule the code below to run every day at the same time.  Midnight ET?

#Get set up
import requests
import json
import pandas as pd
from common.db import create_engine_for_db
engine = create_engine_for_db()
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

from bs4 import BeautifulSoup
#from bs4 import Beautifulsoup4
import requests

#from tqdm import tqdm_notebook as tqdm
import time

In [494]:
def get_data(c,start='20191014',end='20191015'):
    url = 'https://coinmarketcap.com/currencies/'+c+'/historical-data/?start='+start+'&end='+end
    url=url.replace(' ','-')
    content = requests.get(url).content
    soup = BeautifulSoup(content,'html.parser')

    #         time.sleep(1)
#11.8 this stopped working so i switched it to find tbody
#    table = soup.find('table', {'class': 'table'})
    table = soup.find('tbody')
    data = [[td.text.strip() for td in tr.findChildren('td')] 
            for tr in table.findChildren('tr')]
    df = pd.DataFrame(data)
#11.8 removed this line because i switched the find
#    df.drop(df.index[0], inplace=True) # first row is empty
    df[0] =  pd.to_datetime(df[0]) # date
    for i in range(1,7):
        df[i] = pd.to_numeric(df[i].str.replace(",","").str.replace("-","")) # some vol is missing and has -
    df.columns = ['Date','Open','High','Low','Close','Volume','Market Cap']
    df['Name']=c
    return df

In [495]:
df_total=pd.DataFrame() 
df_list=[]
df_tmp=[]
df_err=[]
cnt=0
#all_slugs = pd.read_csv('redo.csv', delimiter = ',')
all_slugs = pd.read_csv('hist_10m_slugs_alive.csv', delimiter = ',')
# for testing  all_slugs = pd.read_csv('slugs_test100.csv', delimiter = ',')
#just for testing - please remove
#all_slugs = all_slugs.head(10)
dt = str(int(time.strftime("%Y%m%d"))-1)
for index, c in all_slugs.iterrows():
    if c['slug'] =='crypterium':#xyo-network
        print('skip')
    else:
        try:
            df_tmp=get_data(c['slug'], dt, dt)
            df_list.append(df_tmp)
            time.sleep(7)
            if cnt ==1:
                print('first one done')
        except:
            print('failed to parse for :%s'%(c['slug']))
            time.sleep(60)
            try:
                df_tmp=get_data(c['slug'], dt, dt)
                df_list.append(df_tmp)
                time.sleep(5)
                print('retry worked for :%s'%(c['slug']))
            except:
                df_err.append(c['slug'])
               # time.sleep(60)                
        cnt = cnt+1
        if cnt%100 == 0:
            print('100 done')
            time.sleep(30)
df_total=pd.concat(df_list)
df_total=df_total.sort_values(by=['Name','Date']).reset_index()
df_total
print(len(df_list))

failed to parse for :ripple
failed to parse for :monero
retry worked for :monero
failed to parse for :paxos-standard-token
failed to parse for :dai
failed to parse for :theta-token
100 done
failed to parse for :zencash
failed to parse for :stasis-eurs
failed to parse for :byteball
failed to parse for :request-network
skip
100 done
failed to parse for :safe-exchange-coin
failed to parse for :republic-protocol
failed to parse for :ethos
failed to parse for :e-dinar-coin
failed to parse for :ethlend
failed to parse for :dew
failed to parse for :tokencard
failed to parse for :wabi
failed to parse for :poa-network
failed to parse for :ttc-protocol
failed to parse for :scroll
100 done
failed to parse for :xtrabytes
failed to parse for :aurumcoin
failed to parse for :faircoin
failed to parse for :coss
failed to parse for :pascal-coin
failed to parse for :internet-node-token
failed to parse for :linda
failed to parse for :daps-token
failed to parse for :ion
failed to parse for :bitcoin-green
f

In [496]:
df_total.head(3)

,index,Date,Open,High,Low,Close,Volume,Market Cap,Name
0,0,2019-12-09,0.036038,0.037010,0.033395,0.034943,3217.78,1401814.0,0chain
1,0,2019-12-09,0.238210,0.238786,0.230645,0.232161,12268346.00,140323012.0,0x
2,0,2019-12-09,0.000029,0.000029,0.000028,0.000028,0.00,753290.0,1337coin


In [497]:
df_total.to_csv('yesterday.csv', index=False)
df=df_total

In [498]:
#set up rank
#load up a ranking
df['ranknow']=0
df = df.sort_values(by=['Date', 'Market Cap'] , ascending=[1, 0])
j=1
last_date=''
i=0
for index, row in df.iterrows():
    if (row['Date'] == last_date):
        df['ranknow'].iloc[i]=j
        j = j+1
    else:
        last_date = row['Date']
        j=1
        df['ranknow'].iloc[i]=j
        j= j+1
    i=i+1
        
print('done')

done


In [499]:
#get audit key
sql = '''
    select max(audit_key)
    from  market_data.daily_tbl
'''
seq = pd.read_sql(sql, con=engine)
seq = seq + 1
df['audit_key'] = seq['max(audit_key)'][0]

In [501]:
#Load to database

df.to_sql('raw_kaggle_yesterday', schema='market_data', con=engine,   if_exists='replace',index=False)

In [502]:


sql = '''   SELECT distinct date
      FROM raw_kaggle_yesterday
'''
dates = pd.read_sql(sql, con=engine)


In [504]:
#pull top twenty 
sql = '''INSERT INTO  leaders20_signal_hist
    SELECT 
    t2.id,
    t1.DATE,
    ((t1.close - old5.close)/old5.close)/5 AS lead5, 
    ((t1.close - old20.close)/old20.close)/20 AS lead20, 
    ((t1.close - old30.close)/old30.close)/30 AS lead30, 
    ((t1.close - old60.close)/old60.close)/60 AS lead60, 
     t1.audit_key
    FROM raw_kaggle_yesterday t1
    join leaders20_tbl t2 ON (t1.name  = t2.id)
    LEFT join    daily_tbl old5  on ( t1.name = old5.id AND DATE_ADD(t1.DATE, INTERVAL -5 DAY)= old5.date )
    LEFT join    daily_tbl old20  on ( t1.name = old20.id AND DATE_ADD(t1.DATE, INTERVAL -20 DAY)= old20.date )
    LEFT join    daily_tbl old30  on ( t1.name = old30.id AND DATE_ADD(t1.DATE, INTERVAL -30 DAY)= old30.date )
    LEFT join    daily_tbl old60  on ( t1.name = old60.id AND DATE_ADD(t1.DATE, INTERVAL -60 DAY)= old60.date )
'''
engine.execute(sql)

This script only works for 1 date per day. It will pull the data by date.  However, it applies 1 value to the entire data set, so it would apply the last days value.

In [505]:
d = {'date': [], 'lead5': []}
updt = pd.DataFrame(data=d)
i=0
for index, row in dates.iterrows():
    dt_test = row['date']
    
    sql = '''   SELECT AVG(dd.lead5) as median_val
    FROM (
    SELECT d.lead5, DATE, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
      FROM leaders20_signal_hist d, (SELECT @rownum:=0) r
      WHERE d.lead5 is NOT NULL
      and d.date = %(dt)s
      ORDER BY d.lead5
    ) as dd
    WHERE dd.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) ) '''
    
    lead5 = pd.read_sql(sql, con=engine,  params={'dt': dt_test})
    df['lead5'] = lead5['median_val'][0]

    sql = '''   SELECT AVG(dd.lead20) as median_val
    FROM (
    SELECT d.lead20, DATE, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
      FROM leaders20_signal_hist d, (SELECT @rownum:=0) r
      WHERE d.lead20 is NOT NULL
      and d.date = %(dt)s
      ORDER BY d.lead20
    ) as dd
    WHERE dd.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) ) '''
    
    lead20 = pd.read_sql(sql, con=engine,  params={'dt': dt_test})
    df['lead20'] = lead20['median_val'][0]
    
    
    sql = '''   SELECT AVG(dd.lead30) as median_val
    FROM (
    SELECT d.lead30, DATE, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
      FROM leaders20_signal_hist d, (SELECT @rownum:=0) r
      WHERE d.lead30 is NOT NULL
      and d.date = %(dt)s
      ORDER BY d.lead30
    ) as dd
    WHERE dd.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) ) '''
    
    lead30 = pd.read_sql(sql, con=engine,  params={'dt': dt_test})
    df['lead30'] = lead30['median_val'][0]
    
    
    sql = '''   SELECT AVG(dd.lead60) as median_val
    FROM (
    SELECT d.lead60, DATE, @rownum:=@rownum+1 as `row_number`, @total_rows:=@rownum
      FROM leaders20_signal_hist d, (SELECT @rownum:=0) r
      WHERE d.lead60 is NOT NULL
      and d.date = %(dt)s
      ORDER BY d.lead60
    ) as dd
    WHERE dd.row_number IN ( FLOOR((@total_rows+1)/2), FLOOR((@total_rows+2)/2) ) '''
    
    lead60 = pd.read_sql(sql, con=engine,  params={'dt': dt_test})
    df['lead60'] = lead60['median_val'][0]
        
    
    updt = updt.append({'date':  dt_test, 'lead5': lead5['median_val'][0], 'lead20': lead20['median_val'][0], 'lead30': lead30['median_val'][0], 'lead60': lead60['median_val'][0]}, ignore_index=True)
    i=i+1
print('done')
print(i)

done
1


In [286]:
#Only for history
#updt.to_sql('updt_hist_leads', schema='market_data', con=engine,   if_exists='append',index=False)

In [506]:
df.head(3)

,index,Date,Open,High,Low,Close,Volume,Market Cap,Name,ranknow,audit_key,lead5,lead20,lead30,lead60
81,0,2019-12-09,7561.80,7618.09,7365.990000,7400.90,1.787202e+10,1.339040e+11,bitcoin,1,58,0.001606,-0.007614,-0.00755,-0.002011
271,0,2019-12-09,151.22,151.91,147.620000,148.23,6.744207e+09,1.613563e+10,ethereum,2,58,0.001606,-0.007614,-0.00755,-0.002011
702,0,2019-12-09,1.01,1.01,0.997568,1.00,1.928321e+10,4.125515e+09,tether,3,58,0.001606,-0.007614,-0.00755,-0.002011


In [507]:
#Load to database

df.to_sql('raw_kaggle_yesterday', schema='market_data', con=engine,   if_exists='replace',index=False)

In [508]:
#run this load on the database because we need to get the name and symbol from the metadata table.
sql = '''
    INSERT INTO daily_tbl
    SELECT 
    t2.id,
    t1.DATE,
      t2.symbol,
    t2.name,
    t1.ranknow,
    t1.Open,
    t1.high,
    t1.low,
    t1.close,
    t1.volume,
    t1.`Market Cap`,
    t1.high-t1.low, 
    (t1.close-t1.low)/(t1.high-t1.low),
    ((t1.close - old30.close)-1)/30, 
    ((t1.close - old60.close)-1)/60, 
    ((t1.close - old90.close)-1)/90, 
    ((t1.close - old180.close)-1)/180,
    t1.lead5,
    t1.lead20,
    t1.lead30,
    t1.lead60,
    t1.audit_key
    FROM raw_kaggle_yesterday t1
    join coin_metadata_tbl2 t2 ON (t1.name  = t2.id)
    LEFT join    daily_tbl old30  on ( t1.name = old30.id AND DATE_ADD(t1.DATE, INTERVAL -30 DAY)= old30.date )
    LEFT join    daily_tbl old60  on ( t1.name = old60.id AND DATE_ADD(t1.DATE, INTERVAL -60 DAY)= old60.date )
    LEFT join    daily_tbl old90  on ( t1.name = old90.id AND DATE_ADD(t1.DATE, INTERVAL -90 DAY)= old90.date )
    LEFT join    daily_tbl old180  on ( t1.name = old180.id AND DATE_ADD(t1.DATE, INTERVAL -180 DAY)= old180.date )
    WHERE t1.high-t1.low>0
'''
engine.execute(sql)

In [509]:
#run this load on the database because we need to get the name and symbol from the metadata table.
sql = '''
    INSERT INTO daily_tbl
    SELECT 
    t2.id,
    t1.DATE,
      t2.symbol,
    t2.name,
    t1.ranknow,
    t1.Open,
    t1.high,
    t1.low,
    t1.close,
    t1.volume,
    t1.`Market Cap`,
    t1.high-t1.low, 
    0,
    ((t1.close - old30.close)-1)/30, 
    ((t1.close - old60.close)-1)/60, 
    ((t1.close - old90.close)-1)/90, 
    ((t1.close - old180.close)-1)/180,
    t1.lead5,
    t1.lead20,
    t1.lead30,
    t1.lead60,
    t1.audit_key
    FROM raw_kaggle_yesterday t1
    join coin_metadata_tbl2 t2 ON (t1.name  = t2.id)
    LEFT join    daily_tbl old30  on ( t1.name = old30.id AND DATE_ADD(t1.DATE, INTERVAL -30 DAY)= old30.date )
    LEFT join    daily_tbl old60  on ( t1.name = old60.id AND DATE_ADD(t1.DATE, INTERVAL -60 DAY)= old60.date )
    LEFT join    daily_tbl old90  on ( t1.name = old90.id AND DATE_ADD(t1.DATE, INTERVAL -90 DAY)= old90.date )
    LEFT join    daily_tbl old180  on ( t1.name = old180.id AND DATE_ADD(t1.DATE, INTERVAL -180 DAY)= old180.date )
    WHERE t1.high-t1.low=0
'''
engine.execute(sql)

In [510]:
sql = '''INSERT INTO raw_kaggle_daily
SELECT DATE, OPEN, high, low, close, volume, `Market Cap`, NAME, ranknow, audit_key
FROM raw_kaggle_yesterday '''
engine.execute(sql)

In [511]:
sql = '''select * from daily_tbl where date = '2019-12-09'  '''
test = pd.read_sql(sql, con=engine,  params={'dt': dt_test})
    

In [512]:
test

,id,date,symbol,name,ranknow,open,high,low,close,volume,...,close_ratio,mom30,mom60,mom90,mom180,market_signal5,market_signal20,market_signal30,market_signal60,audit_key
0,0chain,2019-12-09,ZCN,0chain,421,0.036038,0.037010,0.033395,0.034943,3218,...,0.428216,-0.033599,-0.016882,-0.011174,-0.005837,0.001606,-0.007614,-0.00755,-0.002011,58
1,0x,2019-12-09,ZRX,0x,35,0.238210,0.238786,0.230645,0.232161,12268346,...,0.186218,-0.035728,-0.017335,-0.010318,-0.006111,0.001606,-0.007614,-0.00755,-0.002011,58
2,1337coin,2019-12-09,1337,Elite,504,0.000029,0.000029,0.000028,0.000028,0,...,0.000000,-0.033333,-0.016666,-0.011111,-0.005556,0.001606,-0.007614,-0.00755,-0.002011,58
3,2give,2019-12-09,2GIVE,2GIVE,751,0.000078,0.000078,0.000078,0.000078,0,...,0.000000,-0.033334,-0.016668,-0.011111,-0.005557,0.001606,-0.007614,-0.00755,-0.002011,58
4,ac3,2019-12-09,AC3,AC3,553,0.000832,0.000995,0.000819,0.000978,14,...,0.903409,-0.033351,-0.016672,-0.011117,-0.005570,0.001606,-0.007614,-0.00755,-0.002011,58
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
794,zilla,2019-12-09,ZLA,Zilla,562,0.008152,0.008488,0.007289,0.007476,47278,...,0.155963,-0.033535,-0.016754,-0.011289,-0.005616,0.001606,-0.007614,-0.00755,-0.002011,58
795,zilliqa,2019-12-09,ZIL,Zilliqa,57,0.006529,0.006717,0.006105,0.006106,13471769,...,0.001634,-0.033330,-0.016673,-0.011120,-0.005652,0.001606,-0.007614,-0.00755,-0.002011,58
796,zip,2019-12-09,ZIP,Zipper,357,0.000142,0.000146,0.000130,0.000131,2292601,...,0.062500,-0.033337,-0.016669,-0.011114,-0.005559,0.001606,-0.007614,-0.00755,-0.002011,58
797,zper,2019-12-09,ZPR,ZPER,519,0.000634,0.000636,0.000509,0.000509,2832,...,0.000000,-0.033337,-0.016669,-0.011117,-0.005569,0.001606,-0.007614,-0.00755,-0.002011,58
